In [2]:
from pandas import read_csv
import pandas as pd
from sklearn.model_selection import train_test_split
from tpot import TPOTClassifier
import joblib

d:\gitprojects\machinelearning\auto_ml\tpot\lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [3]:
test = read_csv("NTI_data/test.csv")
train_target = read_csv("NTI_data/train_target.csv")
transactions_test = read_csv("NTI_data/transactions_test.csv")
transactions_train = read_csv("NTI_data/transactions_train.csv")

In [4]:
#Train
#Признаки, связанные с суммой покупки
agg_features=transactions_train.groupby('client_id')['amount_rur'].agg(['sum','mean','max']).reset_index()

#Количество в каждой категории
amount_of_purchase=transactions_train.groupby(['client_id','small_group'])['amount_rur'].count()
counts_train=amount_of_purchase.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
counts_train=counts_train.fillna(0)
counts_train.columns=['small_group_' + str(i) for i in counts_train.columns]

#Сумма в каждой категории
sum_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].sum()
sum_train=sum_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
sum_train=sum_train.fillna(0)
sum_train.columns=['small_group_sum_'+str(i) for i in sum_train.columns]

#Минимум в каждой категории
min_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].min()
min_train=sum_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
min_train=min_train.fillna(0)
min_train.columns=['small_group_min_'+str(i) for i in min_train.columns]

#Максимум в каждой категории
max_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].min()
max_train=sum_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
max_train=max_train.fillna(0)
max_train.columns=['small_group_max_'+str(i) for i in max_train.columns]

#Средняя сумма в каждой категории
avg_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].mean()
avg_train=avg_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
avg_train=avg_train.fillna(0)
avg_train.columns=['small_group_avg_'+str(i) for i in avg_train.columns]

#Соединим все в одну таблицу
counts_train=pd.merge(sum_train,counts_train,on='client_id')
counts_train=pd.merge(avg_train,counts_train,on='client_id')
counts_train=pd.merge(min_train,counts_train,on='client_id')
counts_train=pd.merge(max_train,counts_train,on='client_id')
train=pd.merge(train_target,agg_features,on='client_id')
train=pd.merge(train,counts_train.reset_index(),on='client_id')

In [5]:
train_size = 500
train_data = train.head(train_size)
test_data = train.tail(29000)

In [6]:
train_data

,client_id,bins,sum,mean,max,small_group_max_0,small_group_max_1,small_group_max_2,small_group_max_3,small_group_max_4,...,small_group_192,small_group_193,small_group_195,small_group_196,small_group_197,small_group_198,small_group_199,small_group_200,small_group_202,small_group_203
0,24662,2,30254.011,34.774725,1227.314,0.000,7093.826,64.110,1412.978,729.946,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1046,0,42548.570,52.015367,1210.506,95.346,10920.764,3712.276,607.346,454.812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,34089,2,26842.816,34.325852,782.641,0.000,9969.533,0.000,795.089,440.769,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,34848,1,15773.126,16.160990,109.590,0.000,7985.775,0.733,0.000,248.769,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,47076,3,12488.375,15.929050,541.165,0.000,3907.408,0.000,3821.081,554.640,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,38242,3,115479.744,118.077448,2003.497,0.000,14874.881,5820.340,4904.102,1211.511,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
496,20421,0,23042.660,28.948065,553.658,82.294,7281.356,0.000,1145.022,253.706,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
497,32622,0,27905.055,26.450289,686.930,7.375,8858.236,682.518,801.005,934.249,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
498,25976,2,34749.925,31.082223,1002.917,0.000,6970.800,0.000,984.538,395.687,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
model_finder = TPOTClassifier(generations=5, population_size=50, verbosity=3, random_state=42)
X_train = train_data.drop('bins', 1)
y_train = train_data['bins']

In [8]:
import time
start_time = time.time()

In [9]:
model_finder.fit(X_train, y_train)

32 operators have been imported by TPOT.


Skipped pipeline #36 due to time out. Continuing to the next pipeline.
_pre_test decorator: _random_mutation_operator: num_test=0 Solver lbfgs supports only dual=False, got dual=True.
_pre_test decorator: _random_mutation_operator: num_test=0 '(slice(None, None, None), 0)' is an invalid key.
_pre_test decorator: _random_mutation_operator: num_test=0 Unsupported set of arguments: The combination of penalty='l1' and loss='hinge' is not supported, Parameters: penalty='l1', loss='hinge', dual=True.
Skipped pipeline #78 due to time out. Continuing to the next pipeline.
Skipped pipeline #95 due to time out. Continuing to the next pipeline.

Generation 1 - Current Pareto front scores:
-1	0.516	XGBClassifier(input_matrix, XGBClassifier__learning_rate=1.0, XGBClassifier__max_depth=6, XGBClassifier__min_child_weight=3, XGBClassifier__n_estimators=100, XGBClassifier__nthread=1, XGBClassifier__subsample=0.7000000000000001)
_pre_test decorator: _random_mutation_operator: num_test=0 Unsupported set 

Skipped pipeline #106 due to time out. Continuing to the next pipeline.

Generation 2 - Current Pareto front scores:
-1	0.54	XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=5, XGBClassifier__min_child_weight=14, XGBClassifier__n_estimators=100, XGBClassifier__nthread=1, XGBClassifier__subsample=0.45)
_pre_test decorator: _random_mutation_operator: num_test=0 '(slice(None, None, None), 0)' is an invalid key., XGBClassifier__min_child_weight=14, XGBClassifier__n_estimators=100, XGBClassifier__nthread=1, XGBClassifier__subsample=0.45)
_pre_test decorator: _random_mutation_operator: num_test=0 [17:53:56] C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:543: Check failed: mparam_.num_feature != 0 (0 vs. 0) : 0 feature is supplied.  Are you using raw Booster interface?.
_pre_test decorator: _random_mutation_operator: num_test=0 '(slice(None, None, None), 0)' is an invalid key.
_pre_test decorator: _random_mutation_operator: nu

_pre_test decorator: _random_mutation_operator: num_test=1 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required..

Generation 3 - Current Pareto front scores:
-1	0.5559999999999999	XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=2, XGBClassifier__min_child_weight=14, XGBClassifier__n_estimators=100, XGBClassifier__nthread=1, XGBClassifier__subsample=0.45)
_pre_test decorator: _random_mutation_operator: num_test=0 '(slice(None, None, None), 0)' is an invalid key., XGBClassifier__min_child_weight=14, XGBClassifier__n_estimators=100, XGBClassifier__nthread=1, XGBClassifier__subsample=0.45)
_pre_test decorator: _random_mutation_operator: num_test=0 [18:08:05] C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:543: Check failed: mparam_.num_feature != 0 (0 vs. 0) : 0 feature is supplied.  Are you using raw Booster interface?.
_pre_test decorator: _random_mutation_operator: num_test=0 [18:08:13] C:\User

_pre_test decorator: _random_mutation_operator: num_test=0 Expected n_neighbors <= n_samples,  but n_samples = 50, n_neighbors = 65.
_pre_test decorator: _random_mutation_operator: num_test=0 Expected n_neighbors <= n_samples,  but n_samples = 50, n_neighbors = 92.
_pre_test decorator: _random_mutation_operator: num_test=0 Unsupported set of arguments: The combination of penalty='l1' and loss='hinge' is not supported, Parameters: penalty='l1', loss='hinge', dual=False.
Skipped pipeline #286 due to time out. Continuing to the next pipeline.

Generation 5 - Current Pareto front scores:
-1	0.5559999999999999	XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=2, XGBClassifier__min_child_weight=14, XGBClassifier__n_estimators=100, XGBClassifier__nthread=1, XGBClassifier__subsample=0.45)
-2	0.562	XGBClassifier(ZeroCount(input_matrix), XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=5, XGBClassifier__min_child_weight=14, XGBClassifier__n_estimato

TPOTClassifier(generations=5,
               log_file=<ipykernel.iostream.OutStream object at 0x000001DB05CD0580>,
               population_size=50, random_state=42, verbosity=3)

In [10]:
X_test = test_data.drop('bins', 1)
y_test = test_data['bins']
print(model_finder.score(X_test, y_test))

0.5499310344827586


In [11]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 0 to 29999
Columns: 1015 entries, client_id to small_group_203
dtypes: float64(1013), int64(2)
memory usage: 232.5 MB
None


In [12]:
end_time = time.time()
print("Time spent", end_time - start_time)

Time spent 5280.005608558655


In [13]:
joblib.dump(model_finder, f"tpot_{train_size}.pkl")

PicklingError: Can't pickle <function <lambda> at 0x000001DB05C69EE0>: it's not found as jupyter_client.session.<lambda>